In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain.schema import HumanMessage
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.retrievers import BM25Retriever, EnsembleRetriever
import pickle
from dotenv import load_dotenv
load_dotenv()

True

In [11]:
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH10-Retriever")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH10-Retriever


In [4]:
with open("../../outputs/sample_output_split_documents.pkl", "rb") as f:
    split_documents = pickle.load(f)

In [5]:
split_documents[0].metadata['id']

0

In [6]:
len(split_documents)

109

In [7]:
# 단계 3: 임베딩(Embedding) 생성
embeddings = OpenAIEmbeddings()

In [8]:
vectorstore = FAISS.from_documents(documents=split_documents, embedding=embeddings)

In [9]:
faiss_retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

In [10]:
# bm25 retriever와 faiss retriever를 초기화합니다.
bm25_retriever = BM25Retriever.from_documents(
    split_documents,
)
bm25_retriever.k = 3

In [ ]:
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, faiss_retriever],
    weights=[0.7, 0.3],
)


In [20]:
# 검색 결과 문서를 가져옵니다.
query = "what are risks and challenges in global economy?"
ensemble_result = ensemble_retriever.invoke(query)
bm25_result = bm25_retriever.invoke(query)
faiss_result = faiss_retriever.invoke(query)

# 가져온 문서를 출력합니다.
print("[Ensemble Retriever]")
for i, doc in enumerate(ensemble_result):
    print(f"Content{i}: {doc.page_content}")
    print()

print("[BM25 Retriever]")
for i, doc in enumerate(bm25_result):
    print(f"Content{i}: {doc.page_content}")
    print()

print("[FAISS Retriever]")
for i, doc in enumerate(faiss_result):
    print(f"Content{i}: {doc.page_content}")
    print()

[Ensemble Retriever]
Content0: Finding the right balance in uncertain times 4  
Summary 4  
Recent developments 7  
Projections 14  
Risks and challenges 16  
Policy requirements 18  
FIGURES  
Figure 1. Global growth has remained resilient with front-loading supporting industrial production 8  
Figure 2. United States trade growth soared prior to April but has since weakened 8  
Figure 3. Industrial production and retail sales growth are easing in several major economies 9  
Figure 4. Higher tariff rates are gradually feeding through into the real economy 11  
Figure 5. Labour markets have softened in some countries 12  
Figure 6. Disinflation has stalled in some OECD economies 13  
Figure 7. Heightened risk appetite is coupled with growing concerns about future fiscal risks 14  
Figure 8. Economic growth is projected to weaken in G20 economies 16  
Figure 9. Valuations of technology stocks and crypto-assets are elevated 17  
Figure 10. Policy rates are expected to ease gradually 19



In [14]:
# 단계 6: 프롬프트 생성(Create Prompt)
# 프롬프트를 생성합니다.
prompt = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Answer in Korean.

#Context: 
{context}

#Question:
{question}

#Answer:"""
)

In [15]:
# 단계 7: 언어모델(LLM) 생성
# 모델(LLM) 을 생성합니다.
llm = ChatOpenAI(model_name="gpt-5-mini", temperature=0)
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
chain = (
    {"context": ensemble_retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)
# 체인 실행(Run Chain)
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.
question = "글로벌 경제에서 위험 및 도전 요소는 무엇인가?"
response = chain.invoke(question)
print(response)

주요 위험 및 도전 요소는 다음과 같습니다.

- 무역장벽 증가와 무역 분절화: 관세·비관세 장벽 확대로 교역 위축, 투자·성장 불확실성 확대.  
- 공급망 취약성 및 지정학적 긴장: 핵심 중간재 규격 불일치·공급처 집중으로 공급망 리스크 증대.  
- 물가 재상승(인플레이션) 위험 및 디스인플레이션 정체: 물가 하락세가 멈추거나 다시 상승하면 정책 불확실성 확대.  
- 재정 리스크와 부채지속가능성 문제: 재정 우려 증대 시 경기 대응 여력 축소 및 시장 신뢰 약화.  
- 금융시장 재가격화(자산가치 급변)와 금융불안: 기술주·암호자산 등 밸류에이션 상승, 급격한 조정 시 금융안정 위협.  
- 투자·생산성 저하와 인구구조 문제: 약한 투자·생산성 성장과 고령화로 장기 잠재성장률 하락.  
- 노동시장 제약(인력 부족)과 구조적 문제: 인력 부족·참여율 제약이 성장 제약으로 작용.  
- 정책 불확실성 및 규제·거버넌스 취약성: 무역·통화·재정 정책의 불투명성이 민간 투자·경기 회복을 저해.  
- 기술 변화에 따른 불균형적 영향: AI 등 신기술의 혜택이 노동·산업별로 달라 조정비용·불평등 확대 가능성.

이들 위험이 상호작용하면 성장·물가·금융안정을 동시에 위협할 수 있으므로 국제 협력, 구조개혁, 재정·통화정책의 신중한 운영이 필요합니다.


In [11]:
vectorstore.save_local("faiss_index")